In [81]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import os

import spacy
from bs4 import BeautifulSoup

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [36]:
nlp = spacy.load("en_core_web_sm")
analyser = SentimentIntensityAnalyzer()

tweet_path = os.path.join("../import_data", "tweets", "history_tweets_20-02-2020.csv")

In [87]:
def sentiment_analyzer_scores(sentence, isPrint=True):
    score = analyser.polarity_scores(sentence) 
    if isPrint:
        print("{:-<40} {}".format(sentence, str(score)))
    return score

In [29]:
df = pd.read_csv(tweet_path)
df.columns = ['id', 'created_at', 'favorite_count', 'retweet_count', 'text', 'source',
               'coordinates', 'screen_name', 'created_at', 'followers_count',
               'verified']
df.info()
df.sample(n=5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1290 entries, 0 to 1289
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               1290 non-null   int64 
 1   created_at       1290 non-null   object
 2   favorite_count   1290 non-null   int64 
 3   retweet_count    1290 non-null   int64 
 4   text             1290 non-null   object
 5   source           1290 non-null   object
 6   coordinates      9 non-null      object
 7   screen_name      1290 non-null   object
 8   created_at       1290 non-null   object
 9   followers_count  1290 non-null   int64 
 10  verified         1290 non-null   bool  
dtypes: bool(1), int64(4), object(6)
memory usage: 102.2+ KB


,id,created_at,favorite_count,retweet_count,text,source,coordinates,screen_name,created_at,followers_count,verified
735,1227884417646915585,Thu Feb 13 09:17:12 +0000 2020,0,0,China's Hubei province sees surge in #coronavi...,"<a href=""http://twitter.com/download/android"" ...",NaN,FHxoxo,Thu Sep 20 20:37:11 +0000 2007,1018,False
250,1229564737731596289,Tue Feb 18 00:34:12 +0000 2020,2,1,I believe the best way to protect yourself &am...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,kurade,Sun Jun 21 04:06:44 +0000 2009,526,False
1239,1227062803648794625,Tue Feb 11 02:52:24 +0000 2020,1,1,This makes #coronavirus hard to detect. You ha...,"<a href=""http://twitter.com/download/iphone"" r...",NaN,tsangyoujun,Thu Aug 31 09:49:13 +0000 2017,83,False
810,1227784036434661376,Thu Feb 13 02:38:20 +0000 2020,0,0,How coronavirus (Covid-19) spread day by day #...,"<a href=""http://scottsdigital.com/guerilla-mar...",NaN,scottsdigital,Fri Nov 12 16:46:47 +0000 2010,5778,False
351,1229213019617390592,Mon Feb 17 01:16:36 +0000 2020,4,2,How to you protect yourself from #coronavirusc...,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,anndylian,Wed Mar 30 04:40:36 +0000 2011,929,False


In [33]:
df['text'][1]

'#Coronavirus is having unexpected impacts,including #tradewar : hopefully, there won’t be many more. Taiwan bans Italian pig imports in quarrel over flight ban, East Asia News &amp; Top Stories - The Straits Times https://t.co/5TUPaXQlin'

In [61]:
# do it in entire paragraph
sentiment_analyzer_scores(df['text'][1])

#Coronavirus is having unexpected impacts,including #tradewar : hopefully, there won’t be many more. Taiwan bans Italian pig imports in quarrel over flight ban, East Asia News &amp; Top Stories - The Straits Times https://t.co/5TUPaXQlin {'neg': 0.092, 'neu': 0.793, 'pos': 0.115, 'compound': -0.0258}


{'neg': 0.092, 'neu': 0.793, 'pos': 0.115, 'compound': -0.0258}

In [90]:
# do it in sentence
doc = nlp(df['text'][1])

def get_sentiment(doc):
    total_scores = {'neg':0, 'neu':0, 'pos':0, 'compound':0}
    
    # Convert document to sentences and remove etc symbols
    for sent in doc.sents:    
        sent = str(sent)
        soup = BeautifulSoup(sent,'lxml')
        html_free = soup.get_text()
        scores = sentiment_analyzer_scores(html_free)
        for s in scores.keys():
            total_scores[s] += scores[s]        

    # store scores and average them
    for key in total_scores.keys():
        value = total_scores[key]
        value = value / len(list(doc.sents))
        total_scores[key] = value       
        
    return total_scores
    
get_sentiment(doc)

#Coronavirus is having unexpected impacts,including #tradewar : hopefully, there won’t be many more. {'neg': 0.0, 'neu': 0.816, 'pos': 0.184, 'compound': 0.4019}
Taiwan bans Italian pig imports in quarrel over flight ban, East Asia News & Top Stories - The Straits Times https://t.co/5TUPaXQlin {'neg': 0.148, 'neu': 0.779, 'pos': 0.074, 'compound': -0.4215}


{'neg': 0.074, 'neu': 0.7975, 'pos': 0.129, 'compound': -0.009800000000000003}